In [6]:
import numpy as np
import pandas as pd

# Импортируем классы
from Tools import DateTimeSeriesSplit, Kraken

# Модель и метрика для регрессии
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_percentage_error as MAPE

# ==============================================================
# Пример 1: Регрессия (оставим ваш пример, слегка адаптируем)
# ==============================================================

# Задаем значение для генератора случайных чисел
seed_value = 23
np.random.seed(seed_value)

# creating dataset
c = 12052
X1 = pd.DataFrame()
X1['var_1'] = np.random.rand(c)
X1['var_2'] = np.random.rand(c)
X1['var_3'] = np.random.rand(c)
X1['var_4'] = np.random.rand(c)
X1['var_5'] = np.random.rand(c)
X1['var_6'] = np.random.rand(c)
X1['var_7'] = np.random.rand(c)
X1['var_8'] = np.random.rand(c)
X1['var_9'] = np.random.rand(c)
X1['date'] = pd.date_range(start='1990-01-01', end='2022-12-30', freq='D')

# создаем простую зависимость для первой части данных
y1 = X1['var_1'] + 3 * X1['var_2'] - np.power(X1['var_3'], 1.5)

X2 = pd.DataFrame()
X2['var_1'] = np.random.rand(c)
X2['var_2'] = np.random.rand(c)
X2['var_3'] = np.random.rand(c)
X2['var_4'] = np.random.rand(c)
X2['var_5'] = np.random.rand(c)
X2['var_6'] = np.random.rand(c)
X2['var_7'] = np.random.rand(c)
X2['var_8'] = np.random.rand(c)
X2['var_9'] = np.random.rand(c)
X2['date'] = pd.date_range(start='1990-01-01', end='2022-12-30', freq='D')

# добавляем var_4 в зависимость для второй части
y2 = X2['var_1'] + 3 * X2['var_2'] - np.power(X2['var_3'], 1.5) + 2 * X2['var_4']

X3 = pd.DataFrame()
X3['var_1'] = np.random.rand(c)
X3['var_2'] = np.random.rand(c)
X3['var_3'] = np.random.rand(c)
X3['var_4'] = np.random.rand(c)
X3['var_5'] = np.random.rand(c) 
X3['var_6'] = np.random.rand(c)
X3['var_7'] = np.random.rand(c)
X3['var_8'] = np.random.rand(c)
X3['var_9'] = np.random.rand(c)
X3['date'] = pd.date_range(start='1990-01-01', end='2022-12-30', freq='D')

# добавляем var_4 и var_5 в зависимость для третьей части
y3 = X3['var_1'] + 3 * X3['var_2'] - np.power(X3['var_3'], 1.5) + 2 * X3['var_4'] + 4 * X3['var_5']

X = pd.concat([X1, X2, X3], axis=0)
y = pd.concat([y1, y2, y3], axis=0)

print("Regression dataset shape:", X.shape)

# creating cross validator
cv_datetime = DateTimeSeriesSplit(window=3000, n_splits=3, test_size=300, margin=0)
group_dt = X['date']

# create model for selector
model_reg = LGBMRegressor(max_depth=3, verbosity=-1)

# create list from which we will take vars for model
list_of_vars = list(X.columns)
list_of_vars.remove('date')
if 'index_time' in list_of_vars:
    list_of_vars.remove('index_time')

# create selector
selector_reg = Kraken(
    estimator=model_reg, 
    cv=cv_datetime, 
    metric=MAPE, 
    meta_info_name='example_regression',
    task_type='regression',
    greater_is_better=False,
    comparison_precision= 2 # MAPE => чем меньше, тем лучше  # не влияет для регрессии
)

# get rank dict from vars
selector_reg.get_rank_dict(X, y, list_of_vars, group_dt)

print("Rank dict (regression) top-5:", dict(list(selector_reg.rank_dict.items())[:5]))

# get vars
best_vars_reg = selector_reg.get_vars(X, y, rank_dict=selector_reg.rank_dict, group_dt=group_dt, max_feature_search_rounds=10)
print("Selected vars (regression):", best_vars_reg)

# ==============================================================


Regression dataset shape: (36156, 10)
[get_rank_dict] Начинаем вычисление рангов признаков...
[get_rank_dict] Начинаем обработку фолда 1...
[get_rank_dict] Фолд 1 обработан за 0.22 секунд.
[get_rank_dict] Начинаем обработку фолда 2...
[get_rank_dict] Фолд 2 обработан за 0.21 секунд.
[get_rank_dict] Начинаем обработку фолда 3...
[get_rank_dict] Фолд 3 обработан за 0.20 секунд.
[get_rank_dict] Завершено вычисление рангов. Всего заняло 0.74 секунд.
Rank dict (regression) top-5: {'var_2': 1, 'var_5': 2, 'var_4': 3, 'var_3': 4, 'var_1': 5}
[get_vars] Начинаем процедуру отбора признаков...
[get_vars] Запуск с нуля: ни одной фичи не отобрано, old_scores/best_mean_cv не заданы.
[get_vars] Новый шаг добавления переменной
[get_cross_val_score] Начинаем оценку кросс-валидации для признака 'var_2'...
[get_cross_val_score] Обработка фолда 1 для признака 'var_2'...
[get_cross_val_score] Фолд 1 для 'var_2' обработан за 0.18 секунд.
[get_cross_val_score] Обработка фолда 2 для признака 'var_2'...
[get_